# SVM

El objetivo de este ejercicio es aprender experimentalmente a ajustar un modelo SVM.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Selección de hiperparámetros para SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Tomado del libro 'Python Data Science Handbook' de Jake VanderPlas
def plot_svc_decision_function(model, ax=None, plot_support=True, levels=[-1, 0, 1], linestyles=['--', '-', '--']):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=levels, alpha=0.5,
               linestyles=linestyles)
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

Seleccionar los parámetros $C$ y $\gamma$ requeridos por el kernel gaussiano.

La selección de los modelos de un parámetro se realiza por medio de *cross validation*. Para ello se reserva una muestra aleatoria del conjunto de datos que pueda servir para estimar el grado de generalización de los modelos a evaluar.

Para este caso recibimos directamente el conjunto de entrenamiento `(X, y)` y el conjunto de validación `(Xval, yval)`.

In [ ]:
dataP2_train = pd.read_csv('svmdatatrain.csv', names=['x1', 'x2', 'y'])
X = dataP2_train[['x1', 'x2']]
y = dataP2_train['y']

dataP2_val = pd.read_csv('svmdataval.csv', names=['x1', 'x2', 'y'])
Xval = dataP2_val[['x1', 'x2']]
yval = dataP2_val['y']

plt.title('Conjunto de datos Pregunta 2')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X['x1'], X['x2'], c=y, s=10, cmap='prism', label='Conjunto de entrenamiento')
plt.scatter(Xval['x1'], Xval['x2'], c=yval, marker='x', s=20, cmap='prism', label='Conjunto de validacion')
plt.show()

Realizaremos también la normalización de los datos. Presta atención a que la normalización de los datos de validación $X_{val}$ se debe realizar con los parámetros obtenidos del conjunto de entrenamiento $X$.

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
Xval_scaled = scaler.transform(Xval)

Comparemos la exactitud en los conjuntos de entrenamiento y validación para un modelo con $C=200$ y $\gamma = 50$

In [ ]:
model = SVC(kernel='rbf', C=200, gamma=50)
model.fit(X_scaled, y)

score_train = model.score(X_scaled, y)
score_val = model.score(Xval_scaled, yval)

print ('Exactitud en el conjunto de entrenamiento: %0.4f' % score_train)
print ('Exactitud en el conjunto de validación: %0.4f' % score_val)


Si bien se tiene una exactitud muy alta en el conjunto de entrenamiento, la exactitud es bastante menor en el conjunto de validación. Este es un síntoma claro de sobreajuste *(overfitting)*. Visualicemos lo que está ocurriendo.

In [ ]:
# Visualización
plt.title('SVM con kernel gaussiano (C=200, gamma=50) (Conjunto de datos 3 normalizado)')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X_scaled[:,0], X_scaled[:,1], c=y, s=10, cmap='prism', label='Conjunto de entrenamiento')
plt.scatter(Xval_scaled[:,0], Xval_scaled[:,1], c=yval, marker='x', s=20, cmap='prism', label='Conjunto de validacion')
plot_svc_decision_function(model, plot_support=False, levels=[0], linestyles=['-']) 

Podemos apreciar visualmente que la generalización es deficiente porque el modelo está *"memorizando"* la ubicación de los valores atípicos. Este no es un comportamiento deseable.

Para determinar valores $C$ y $\gamma$ que brinden una mejor generalización, es necesario probar con diferentes valores, ajustando el modelo en el conjunto de entrenamiento y probando su desempeño en el conjunto de validación.

In [ ]:
Cs = np.logspace(-2,2,9)  # ~ [0.01, 0.03, 0.1, ..., 100]
gammas = np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]

mejor_modelo = None
mejor_score = 0
for C in Cs:
    for gamma in gammas:
        
        model = SVC(kernel='rbf', C=C, gamma=gamma)
        model.fit(X_scaled, y)
        
        score_val = model.score(Xval_scaled, yval)
        
        if score_val > mejor_score:
            mejor_score = score_val
            mejor_modelo = model

print ('Mejor valor de C: %0.4f' % mejor_modelo.get_params()['C'])
print ('Mejor valor de gamma: %0.4f' % mejor_modelo.get_params()['gamma'])
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % mejor_modelo.score(X_scaled, y))
print ('Exactitud en el conjunto de validación: %0.4f' % mejor_modelo.score(Xval_scaled, yval))

# Visualización
plt.title('SVM con kernel gaussiano (Conjunto de datos 3)')
plt.margins(x=0, y=0)
plt.scatter(X_scaled[:,0], X_scaled[:,1], c=y, s=10, cmap='prism', label='Conjunto de entrenamiento')
plt.scatter(Xval_scaled[:,0], Xval_scaled[:,1], c=yval, marker='x', s=20, cmap='prism', label='Conjunto de validacion')
plot_svc_decision_function(mejor_modelo, plot_support=False, levels=[0], linestyles=['-']) 


### Selección de hiperparámetros para SVM con GridSearchCV

Probar la funcionalidad de GridSearchCV de la librería "sklearn" para identificar los mejores hiperparámetros para el algoritmo SVM sobre el conjunto de entrenamiento "train". Luego, analice el resultado en el conjunto de validación "valid".
<br>http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
<br>http://scikit-learn.org/stable/modules/grid_search.html#multimetric-grid-search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C':np.logspace(-2,2,9), 'gamma':np.logspace(-4,4,9)}

model = SVC()
gridsearchcv = GridSearchCV(model, param_grid, cv=5)

gridsearchcv.fit(X_scaled, y)

print(gridsearchcv.best_params_)
print(gridsearchcv.best_score_)

In [ ]:
val_model = SVC(kernel='rbf', C=10, gamma=1)
val_model.fit(X_scaled, y)

validation_score = val_model.score(Xval_scaled, yval)
print(validation_score)